In [1]:
from pathlib import Path
from sqlalchemy import create_engine
import sqlite3
import pandas as pd

Path('database.db').touch()
conn = sqlite3.connect('database.db')
c = conn.cursor()
c.execute('''CREATE TABLE summer_medals (Year INTEGER,	
                                     City TEXT,
                                     Sport TEXT,
                                     Discipline TEXT,
                                     Athlete TEXT,
                                     Country TEXT,	
                                     Gender TEXT,
                                     Event TEXT,
                                     Medal TEXT);''')
pd.read_csv("summer.csv").to_sql("summer_medals", conn, if_exists='append', index = False)

engine = create_engine("sqlite:///database.db")
engine.table_names()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)


['summer_medals']

In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///database.db")
engine.table_names()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  This is separate from the ipykernel package so we can avoid doing imports until


['summer_medals']

### ROW_NUMBER()



In [15]:
pd.read_sql_query('''
SELECT Year, Event, Country, 
       ROW_NUMBER() OVER() AS Row_N
FROM summer_medals
WHERE Medal = 'Gold';''', engine).head()

,Year,Event,Country,Row_N
0,1896,100M Freestyle,HUN,1
1,1896,100M Freestyle For Sailors,GRE,2
2,1896,1200M Freestyle,HUN,3
3,1896,400M Freestyle,AUT,4
4,1896,100M,USA,5


### Numbering rows
The simplest application for window functions is numbering rows. Numbering rows allows you to easily fetch the `n`th row. For example, it would be very difficult to get the `35`th row in any given table if you didn't have a column with each row's number.

In [4]:
pd.read_sql_query('''
SELECT *,
       ROW_NUMBER() OVER() AS Row_N
FROM Summer_Medals;''', engine).head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Row_N
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold,1
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver,2
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze,3
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold,4
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver,5


### Numbering Olympic games in ascending order

The Summer Olympics dataset contains the results of the games between 1896 and 2012. The first Summer Olympics were held in 1896, the second in 1900, and so on. What if you want to easily query the table to see in which year the 13th Summer Olympics were held? You'd need to number the rows for that.

* Assign a number to each year in which Summer Olympic games were held.

In [5]:
pd.read_sql_query('''
SELECT Year,
       ROW_NUMBER() OVER() AS Row_N
FROM (SELECT DISTINCT year
             FROM Summer_Medals
             ORDER BY Year) ''', engine).head()

,year,Row_N
0,1896,1
1,1900,2
2,1904,3
3,1908,4
4,1912,5


### LEAD and LAG

* `LAG(col_name, number) OVER(...)`

In [6]:
pd.read_sql_query('''
SELECT year, country AS champion
FROM summer_medals
WHERE year IN (1996, 2000, 2004, 2008, 2012) AND gender = 'Men' AND Medal = 'Gold' AND Event = 'Discus Throw'; ''', engine)

,Year,champion
0,1996,GER
1,2000,LTU
2,2004,LTU
3,2008,EST
4,2012,GER


In [10]:
pd.read_sql_query('''
WITH Discus_Gold AS 
(SELECT year, country AS champion
FROM summer_medals
WHERE year IN (1996, 2000, 2004, 2008, 2012) AND gender = 'Men' AND Medal = 'Gold' AND Event = 'Discus Throw')

SELECT year, champion,
       LAG(champion) OVER(ORDER BY year) AS previous_champion,
       LAG(champion, 2) OVER(ORDER BY year) AS previous_two_champion,
       LAG(champion, 3) OVER(ORDER BY year) AS previous_three_champion
FROM Discus_Gold
ORDER BY year; ''', engine)

,year,champion,previous_champion,previous_two_champion,previous_three_champion
0,1996,GER,None,None,None
1,2000,LTU,GER,None,None
2,2004,LTU,LTU,GER,None
3,2008,EST,LTU,LTU,GER
4,2012,GER,EST,LTU,LTU


### **Numbering Olympic games in descending order**

You've already numbered the rows in the Summer Medals dataset. What if you need to reverse the row numbers so that the most recent Olympic games' rows have a lower number?

* Assign a number to each year in which Summer Olympic games were held so that rows with the most recent years have lower row numbers.

In [11]:
pd.read_sql_query('''
SELECT Year,
       ROW_NUMBER() OVER (ORDER BY Year DESC) AS Row_N
FROM (
      SELECT DISTINCT Year
      FROM Summer_Medals
    ) AS Years
ORDER BY Year DESC;''', engine)

,Year,Row_N
0,2012,1
1,2008,2
2,2004,3
3,2000,4
4,1996,5
5,1992,6
6,1988,7
7,1984,8
8,1980,9
9,1976,10


### **Numbering Olympic athletes by medals earned**

Row numbering can also be used for ranking. For example, numbering rows and ordering by the count of medals each athlete earned in the OVER clause will assign 1 to the highest-earning medalist, 2 to the second highest-earning medalist, and so on.

* For each athlete, count the number of medals he or she has earned.

In [13]:
pd.read_sql_query('''
SELECT athlete,
       COUNT(medal) AS Medals
FROM Summer_Medals
GROUP BY Athlete
ORDER BY Medals DESC;''', engine).head()

,Athlete,Medals
0,PHELPS Michael,22
1,LATYNINA Larisa,18
2,ANDRIANOV Nikolay,15
3,SHAKHLIN Boris,13
4,ONO Takashi,13


* Having wrapped the previous query in the `Athlete_Medals` CTE, rank each athlete by the number of medals they've earned.

In [15]:
pd.read_sql_query('''
WITH Athlete_Medals AS (SELECT Athlete, 
                               COUNT(*) AS Medals
                        FROM Summer_Medals
                        GROUP BY Athlete)

SELECT Athlete,
       ROW_NUMBER() OVER (ORDER BY Medals DESC) AS Row_N
FROM Athlete_Medals
ORDER BY Medals DESC;''', engine).head()

,Athlete,Row_N
0,PHELPS Michael,1
1,LATYNINA Larisa,2
2,ANDRIANOV Nikolay,3
3,MANGIAROTTI Edoardo,4
4,ONO Takashi,5


### **Reigning weightlifting champions**

A reigning champion is a champion who's won both the previous and current years' competitions. To determine if a champion is reigning, the previous and current years' results need to be in the same row, in two different columns.

* Return each year's gold medalists in the Men's 69KG weightlifting competition.

In [17]:
pd.read_sql_query('''
SELECT year,
       country AS champion
FROM Summer_Medals
WHERE Discipline = 'Weightlifting' 
AND Event = '69KG' 
AND Gender = 'Men' 
AND Medal = 'Gold';''', engine)

,Year,champion
0,2000,BUL
1,2004,CHN
2,2008,CHN
3,2012,CHN


* Having wrapped the previous query in the `Weightlifting_Gold` CTE, get the previous year's champion for each year.

In [19]:
pd.read_sql_query('''
WITH Weightlifting_Gold AS (SELECT Year,
                                   Country AS champion
                            FROM Summer_Medals
                            WHERE Discipline = 'Weightlifting' 
                            AND Event = '69KG' 
                            AND Gender = 'Men' 
                            AND Medal = 'Gold')

SELECT Year, Champion,
       LAG(Champion) OVER(ORDER BY year) AS Last_Champion
FROM Weightlifting_Gold
ORDER BY Year;''', engine)

,Year,champion,Last_Champion
0,2000,BUL,None
1,2004,CHN,BUL
2,2008,CHN,CHN
3,2012,CHN,CHN


# PARTITION BY()

จากโค้ดด้านล่างจะเห็นว่าถ้าไม่ใช้ partition  จะได้ query ที่ไม่ถูก จะเห็นว่า บรรทัดที่ 5 มันไปเอาค่า lag จาก บรรทัดที่ 4 มา ซึ่งเป็นของการแข่งขันรายการอื่น

In [26]:
pd.read_sql_query('''
WITH Discus_Gold AS 
(SELECT year, event, country AS champion
FROM summer_medals
WHERE year IN (1996, 2000, 2004, 2008, 2012) AND gender = 'Men' AND Medal = 'Gold' 
AND Event IN ('Discus Throw', 'Triple Jump'))

SELECT year, event, champion,
       LAG(champion) OVER(ORDER BY event, year) AS previous_champion,
       ROW_NUMBER() OVER(ORDER BY event, year) AS Row_N
FROM Discus_Gold
ORDER BY event, year; ''', engine)

,year,event,champion,previous_champion,Row_N
0,1996,Discus Throw,GER,None,1
1,2000,Discus Throw,LTU,GER,2
2,2004,Discus Throw,LTU,LTU,3
3,2008,Discus Throw,EST,LTU,4
4,2012,Discus Throw,GER,EST,5
5,1996,Triple Jump,USA,GER,6
6,2000,Triple Jump,GBR,USA,7
7,2004,Triple Jump,SWE,GBR,8
8,2008,Triple Jump,POR,SWE,9
9,2012,Triple Jump,USA,POR,10


จะเห็นว่าโค้ดด้านล่างแก้ไขแล้ว 

In [28]:
pd.read_sql_query('''
WITH Discus_Gold AS 
(SELECT year, event, country AS champion
FROM summer_medals
WHERE year IN (1996, 2000, 2004, 2008, 2012) AND gender = 'Men' AND Medal = 'Gold' 
AND Event IN ('Discus Throw', 'Triple Jump'))

SELECT year, event, champion,
       LAG(champion) OVER(PARTITION BY event ORDER BY event, year) AS previous_champion,
       ROW_NUMBER()  OVER(PARTITION BY event ORDER BY event, year) AS Row_N
FROM Discus_Gold
ORDER BY event, year; ''', engine)

,year,event,champion,previous_champion,Row_N
0,1996,Discus Throw,GER,None,1
1,2000,Discus Throw,LTU,GER,2
2,2004,Discus Throw,LTU,LTU,3
3,2008,Discus Throw,EST,LTU,4
4,2012,Discus Throw,GER,EST,5
5,1996,Triple Jump,USA,None,1
6,2000,Triple Jump,GBR,USA,2
7,2004,Triple Jump,SWE,GBR,3
8,2008,Triple Jump,POR,SWE,4
9,2012,Triple Jump,USA,POR,5


###  Reigning champions by gender

You've already fetched the previous year's champion for one event. However, if you have multiple events, genders, or other metrics as columns, you'll need to split your table into partitions to avoid having a champion from one event or gender appear as the previous champion of another event or gender.

* Return the previous champions of each year's event by gender.

In [29]:
pd.read_sql_query('''
WITH Tennis_Gold AS (SELECT DISTINCT Gender, Year, Country
                     FROM Summer_Medals
                     WHERE Year >= 2000 
                     AND Event = 'Javelin Throw' 
                     AND Medal = 'Gold')

SELECT Gender, Year, Country AS Champion,
       LAG(Country) OVER (PARTITION BY gender ORDER BY year) AS Last_Champion
FROM Tennis_Gold
ORDER BY Gender, Year;''', engine)

,Gender,Year,Champion,Last_Champion
0,Men,2000,CZE,None
1,Men,2004,NOR,CZE
2,Men,2008,NOR,NOR
3,Men,2012,TTO,NOR
4,Women,2000,NOR,None
5,Women,2004,CUB,NOR
6,Women,2008,CZE,CUB
7,Women,2012,CZE,CZE


### Reigning champions by gender and event

In the previous exercise, you partitioned by `gender` to ensure that data about one `gender` doesn't get mixed into data about the other `gender`. If you have multiple columns, however, partitioning by only one of them will still mix the results of the other columns.

* Return the previous champions of each year's events by `gender` and `event`.

In [30]:
pd.read_sql_query(''' 
WITH Athletics_Gold AS (SELECT DISTINCT Gender, Year, Event, Country
                        FROM Summer_Medals
                        WHERE Year >= 2000 
                        AND Discipline = 'Athletics' 
                        AND Event IN ('100M', '10000M') 
                        AND Medal = 'Gold')

SELECT Gender, Year, Event, Country AS Champion,
       LAG(Country) OVER (PARTITION BY gender, event ORDER BY Year) AS Last_Champion
FROM Athletics_Gold
ORDER BY Event, Gender, Year;''', engine)

,Gender,Year,Event,Champion,Last_Champion
0,Men,2000,10000M,ETH,None
1,Men,2004,10000M,ETH,ETH
2,Men,2008,10000M,ETH,ETH
3,Men,2012,10000M,GBR,ETH
4,Women,2000,10000M,ETH,None
5,Women,2004,10000M,CHN,ETH
6,Women,2008,10000M,ETH,CHN
7,Women,2012,10000M,ETH,ETH
8,Men,2000,100M,USA,None
9,Men,2004,100M,USA,USA


### Row numbers with partitioning
If you run ROW_NUMBER() OVER (PARTITION BY Year ORDER BY Medals DESC) on the following table, what row number would the 2008 Iranian record have?


```
| Year | Country | Medals |
|------|---------|--------|
| 2004 | IRN     | 32     |
| 2004 | LBN     | 17     |
| 2004 | KSA     | 4      |
| 2008 | IRQ     | 29     |
| 2008 | IRN     | 27     |
| 2008 | UAE     | 12     |
```

The answer is 2.  It would be 5 if there is no partition.

### Future gold medalists

Fetching functions allow you to get values from different parts of the table into one row. If you have time-ordered data, you can "peek into the future" with the `LEAD` fetching function. This is especially useful if you want to compare a current value to a future value.

* For each year, fetch the current gold medalist and the gold medalist 3 competitions ahead of the current row.

In [31]:
pd.read_sql_query('''
WITH Discus_Medalists AS (SELECT DISTINCT Year, Athlete
                          FROM Summer_Medals
                          WHERE Medal = 'Gold'
                          AND Event = 'Discus Throw'
                          AND Gender = 'Women'
                          AND Year >= 2000)

SELECT year, Athlete,
       LEAD(Athlete, 3) OVER (ORDER BY year) AS Future_Champion
FROM Discus_Medalists
ORDER BY Year;''', engine)

,Year,Athlete,Future_Champion
0,2000,ZVEREVA Ellina,PERKOVIC Sandra
1,2004,SADOVA Natalya,None
2,2008,BROWN TRAFTON Stephanie,None
3,2012,PERKOVIC Sandra,None


### First athlete by name

It's often useful to get the first or last value in a dataset to compare all other values to it. With absolute fetching functions like `FIRST_VALUE`, you can fetch a value at an absolute position in the table, like its beginning or end.

* Return all athletes and the first athlete ordered by alphabetical order.

In [33]:
pd.read_sql_query('''
WITH All_Male_Medalists AS (SELECT DISTINCT Athlete
                            FROM Summer_Medals
                            WHERE Medal = 'Gold'
                            AND Gender = 'Men')

SELECT Athlete,
       FIRST_VALUE(Athlete) OVER (ORDER BY Athlete) AS First_Athlete
FROM All_Male_Medalists;''', engine).head()

,Athlete,First_Athlete
0,AABYE Edgar,AABYE Edgar
1,AALTONEN Paavo Johannes,AABYE Edgar
2,AAS Thomas Valentin,AABYE Edgar
3,ABALMASAU Aliaksei,AABYE Edgar
4,ABALO Luc,AABYE Edgar


### Last country by name

Just like you can get the first row's value in a dataset, you can get the last row's value. This is often useful when you want to compare the most recent value to previous values.

* Return the `year` and the `city` in which each Olympic games were held.
* Fetch the last city in which the Olympic games were held.

# LAST_VALUE() ต้องมี Frames ด้วย 
* `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`

In [38]:
pd.read_sql_query('''
WITH Hosts AS (SELECT DISTINCT Year, City
               FROM Summer_Medals)

SELECT Year, City,
       LAST_VALUE(City) OVER (ORDER BY year ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS Last_City
FROM Hosts
ORDER BY Year;''', engine)

,Year,City,Last_City
0,1896,Athens,London
1,1900,Paris,London
2,1904,St Louis,London
3,1908,London,London
4,1912,Stockholm,London
5,1920,Antwerp,London
6,1924,Paris,London
7,1928,Amsterdam,London
8,1932,Los Angeles,London
9,1936,Berlin,London


# RANK

* `ROW_NUMBER()` ใส่หมายเลขที่เรียงตามลำดับตลอด แม้ค่าจะซ้ำ
* `RANK()` ถ้าค่าซ้ำ จะใส่เลขเดียวกัน จะข้ามลำดับไป ถ้ามีตัวซ้ำ
* `DENSE_RANK()` คล้ายๆ RANK

In [42]:
pd.read_sql('''
SELECT Country, COUNT(DISTINCT Year) AS Games
FROM Summer_Medals
WHERE Country IN ('GBR', 'DEN', 'FRA',
                  'ITA', 'AUT', 'BEL',
                  'NOR', 'POL', 'ESP')
GROUP BY Country
ORDER BY Games DESC;''', engine)

,Country,Games
0,GBR,27
1,FRA,26
2,DEN,26
3,ITA,25
4,BEL,24
5,AUT,24
6,NOR,22
7,POL,20
8,ESP,18


ถ้าใช้ RANK จะเป็นอย่างไร

In [46]:
pd.read_sql('''
WITH CTE AS (SELECT Country, COUNT(DISTINCT Year) AS Games
             FROM Summer_Medals
             WHERE Country IN ('GBR', 'DEN', 'FRA',
                               'ITA', 'AUT', 'BEL',
                               'NOR', 'POL', 'ESP')
             GROUP BY Country
             ORDER BY Games DESC)

SELECT Country, Games,
       ROW_NUMBER() OVER (ORDER BY Games DESC) AS Row_N,
       RANK() OVER (ORDER BY Games DESC) AS Rank_N,
       DENSE_RANK() OVER (ORDER BY Games DESC) AS DRank_N
FROM CTE
ORDER BY Games DESC, Country;''', engine)

,Country,Games,Row_N,Rank_N,DRank_N
0,GBR,27,1,1,1
1,DEN,26,3,2,2
2,FRA,26,2,2,2
3,ITA,25,4,4,3
4,AUT,24,6,5,4
5,BEL,24,5,5,4
6,NOR,22,7,7,5
7,POL,20,8,8,6
8,ESP,18,9,9,7


## ลองมาดู RANK อีกตัวอย่าง

In [48]:
pd.read_sql('''
SELECT Country, Athlete, COUNT(*) AS Medals
FROM Summer_Medals
WHERE Country IN ('CHN', 'RUS') AND Year = 2012
GROUP BY Country, Athlete
HAVING COUNT(*) > 1
ORDER BY Country, Medals DESC;''', engine)

,Country,Athlete,Medals
0,CHN,SUN Yang,4
1,CHN,ZOU Kai,3
2,CHN,WANG Hao,3
3,CHN,GUO Shuang,3
4,CHN,ZHAO Yunlei,2
5,CHN,ZHANG Jike,2
6,CHN,YE Shiwen,2
7,CHN,WU Minxia,2
8,CHN,SUN Yujie,2
9,CHN,QIN Kai,2


In [51]:
pd.read_sql('''
WITH CTE AS (SELECT Country, Athlete, COUNT(*) AS Medals
             FROM Summer_Medals
             WHERE Country IN ('CHN', 'RUS') AND Year = 2012
             GROUP BY Country, Athlete
             HAVING COUNT(*) > 1
             ORDER BY Country, Medals DESC)

SELECT Country, Athlete, Medals,
       DENSE_RANK() OVER (ORDER BY Medals DESC) AS Rank_N
       FROM CTE
       ORDER BY Country, Medals DESC;''', engine)

,Country,Athlete,Medals,Rank_N
0,CHN,SUN Yang,4,1
1,CHN,ZOU Kai,3,2
2,CHN,WANG Hao,3,2
3,CHN,GUO Shuang,3,2
4,CHN,ZHAO Yunlei,2,3
5,CHN,ZHANG Jike,2,3
6,CHN,YE Shiwen,2,3
7,CHN,WU Minxia,2,3
8,CHN,SUN Yujie,2,3
9,CHN,QIN Kai,2,3


### เหมือนจะถูกแต่ไม่ถูก เพราะ

```
	Country	Athlete  	  Medals  Rank_N
18	RUS	MUSTAFINA Aliya	  4	1
19	RUS	ZAKHAROV Ilya	    2	3
20	RUS	ZABELINSKAYA Olga	2	3
```

แทนที่จะเป็นอันดับ 2 แต่ข้ามมาเป็นอันดับ 3 เพราะไม่ใส่ `PARTITION BY` เลยกลายเป็นการดู `RANK` เทียบกับข้อมูลทั้งตาราง

* นั่นคือถ้ามีกลุ่มของข้อมูลที่ต้องการจะจัดลำดับ ก็ต้องจัดการ PARTITION เสียก่อน

### Ranking athletes by medals earned

In chapter 1, you used `ROW_NUMBER` to rank athletes by awarded medals. However, `ROW_NUMBER` assigns different numbers to athletes with the same count of awarded medals, so it's not a useful ranking function; if two athletes earned the same number of medals, they should have the same rank.

* Rank each `athlete` by the number of `medals` they've earned -- the higher the count, the higher the rank -- with identical numbers in case of identical values.

In [53]:
pd.read_sql('''
WITH Athlete_Medals AS (SELECT Athlete, COUNT(*) AS Medals
                        FROM Summer_Medals
                        GROUP BY Athlete)

SELECT Athlete, Medals,
       RANK() OVER (ORDER BY Medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Medals DESC;''', engine).head()

,Athlete,Medals,Rank_N
0,PHELPS Michael,22,1
1,LATYNINA Larisa,18,2
2,ANDRIANOV Nikolay,15,3
3,MANGIAROTTI Edoardo,13,4
4,ONO Takashi,13,4


### Ranking athletes from multiple countries
In the previous exercise, you used `RANK` to assign rankings to one group of athletes. In real-world data, however, you'll often find numerous groups within your data. Without partitioning your data, one group's values will influence the rankings of the others.

Also, while `RANK` skips numbers in case of identical values, the most natural way to assign rankings is not to skip numbers. If two countries are tied for second place, the country after them is considered to be third by most people.

* Rank each `country`'s athletes by the count of `medals` they've earned -- the higher the count, the higher the rank -- without skipping numbers in case of identical values.

In [54]:
pd.read_sql('''
WITH Athlete_Medals AS (SELECT Country, 
                               Athlete, 
                               COUNT(*) AS Medals
                        FROM Summer_Medals
                        WHERE Country IN ('JPN', 'KOR')
                        AND Year >= 2000
                        GROUP BY Country, Athlete
                        HAVING COUNT(*) > 1)

SELECT Country, Athlete,
       DENSE_RANK() OVER (PARTITION BY Country ORDER BY Medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Country, RANK_N;''', engine)

,Country,Athlete,Rank_N
0,JPN,KITAJIMA Kosuke,1
1,JPN,UCHIMURA Kohei,2
2,JPN,TACHIBANA Miya,3
3,JPN,TAKEDA Miho,3
4,JPN,ICHO Kaori,4
...,...,...,...
69,KOR,OH Yong Ran,4
70,KOR,PARK Jinman,4
71,KOR,PARK Kyung-Mo,4
72,KOR,YOO Yong-Sung,4


# Paging: Splitting data into approximately equal chunks

* `NTILE(n)` แบ่งข้อมูลออกเป็น n ส่วนเท่าๆกัน

ก่อนอื่นมาดู CTE

## NTILE(n) สามารถแบ่งข้อมูลออกเป็น n ส่วนได้ ดังนั้น ถ้าเราเอาข้อมูลมาเรียงลำดับ ก็จะทำให้สามารถแบ่ง ข้อมูลตามเปอร์เซ็นไทล์ได้

In [55]:
pd.read_sql('''
SELECT DISTINCT Discipline
FROM Summer_Medals;''', engine)    

,Discipline
0,Swimming
1,Athletics
2,Cycling Road
3,Cycling Track
4,Fencing
...,...
62,Gymnastics Artistic
63,Gymnastics Rhythmic
64,Modern Pentathlon
65,Beach Volleyball


In [58]:
pd.read_sql_query('''
WITH CTE AS (SELECT DISTINCT Discipline
                     FROM Summer_Medals)

SELECT Discipline, NTILE(15) OVER () AS Page
From CTE
ORDER BY Page;''', engine).head(15)

,Discipline,Page
0,Swimming,1
1,Athletics,1
2,Cycling Road,1
3,Cycling Track,1
4,Fencing,1
5,Artistic G.,2
6,Shooting,2
7,Tennis,2
8,Weightlifting,2
9,Wrestling Gre-R,2


## แบ่งข่อมูลตามลำดับ


In [61]:
pd.read_sql_query('''
WITH Country_Medals AS (SELECT Country, COUNT(*) AS Medals
                        FROM Summer_Medals
                        GROUP BY Country)
SELECT Country, Medals,
       NTILE(3) OVER (ORDER BY Medals DESC) AS Third
FROM Country_Medals;''', engine)

,Country,Medals,Third
0,USA,4585,1
1,URS,2049,1
2,GBR,1720,1
3,FRA,1396,1
4,GER,1305,1
...,...,...,...
143,SEN,1,3
144,SUD,1,3
145,TGA,1,3
146,TOG,1,3


คำนวณหาจากตารางด้านบน

In [62]:
pd.read_sql_query('''
WITH Country_Medals AS (SELECT Country, COUNT(*) AS Medals
                        FROM Summer_Medals
                        GROUP BY Country),
             Thirds AS (SELECT Country, Medals,
                               NTILE(3) OVER (ORDER BY Medals DESC) AS Third
                        FROM Country_Medals)

SELECT Third,
        ROUND(AVG(Medals), 2) AS Avg_Medals
FROM Thirds
GROUP BY Third
ORDER BY Third''', engine)

,Third,Avg_Medals
0,1,598.74
1,2,22.98
2,3,2.08


### Paging events
There are exactly 666 unique events in the Summer Medals Olympics dataset. If you want to chunk them up to analyze them piece by piece, you'll need to split the events into groups of approximately equal size.

* Split the distinct events into exactly 111 groups, ordered by event in alphabetical order.

In [63]:
pd.read_sql_query('''
WITH Events AS (SELECT DISTINCT Event
                FROM Summer_Medals)
  
SELECT Event,
       NTILE(111) OVER (ORDER BY Event) AS Page
FROM Events
ORDER BY Event;''', engine).head(14)

,Event,Page
0,+ 100KG,1
1,+ 100KG (Heavyweight),1
2,+ 100KG (Super Heavyweight),1
3,+ 105KG,1
4,+ 108KG Total (Super Heavyweight),1
5,+ 110KG Total (Super Heavyweight),1
6,+ 67 KG,2
7,+ 71.67KG (Heavyweight),2
8,+ 72KG (Heavyweight),2
9,+ 73KG (Heavyweight),2


### Top, middle, and bottom thirds

Splitting your data into thirds or quartiles is often useful to understand how the values in your dataset are spread. Getting summary statistics (averages, sums, standard deviations, etc.) of the top, middle, and bottom thirds can help you determine what distribution your values follow.

* Split the athletes into top, middle, and bottom thirds based on their count of medals.

In [64]:
pd.read_sql('''
WITH Athlete_Medals AS (SELECT Athlete, COUNT(*) AS Medals
                        FROM Summer_Medals
                        GROUP BY Athlete
                        HAVING COUNT(*) > 1)
  
SELECT Athlete,
       Medals,
       NTILE(3) OVER(ORDER BY Medals DESC) AS Third
FROM Athlete_Medals
ORDER BY Medals DESC, Athlete ASC;''', engine)

,Athlete,Medals,Third
0,PHELPS Michael,22,1
1,LATYNINA Larisa,18,1
2,ANDRIANOV Nikolay,15,1
3,MANGIAROTTI Edoardo,13,1
4,ONO Takashi,13,1
...,...,...,...
5261,ZVEREVA Ellina,2,3
5262,ZWERVER Ronald,2,3
5263,ZWOLLE Hendrik Jan,2,3
5264,ZYKINA Olesya,2,3


* Return the average of each third.

In [65]:
pd.read_sql_query('''
WITH Athlete_Medals AS (SELECT Athlete, COUNT(*) AS Medals
                        FROM Summer_Medals
                        GROUP BY Athlete
                        HAVING COUNT(*) > 1),
  
             Thirds AS (SELECT Athlete, Medals,
                        NTILE(3) OVER (ORDER BY Medals DESC) AS Third
                        FROM Athlete_Medals)
  
SELECT Third, AVG(Medals) AS Avg_Medals
FROM Thirds
GROUP BY Third
ORDER BY Third;''', engine)

,Third,Avg_Medals
0,1,3.786446
1,2,2.000000
2,3,2.000000


# Aggregate window functions

In [66]:
pd.read_sql_query('''
SELECT Year, 
       COUNT(*) AS Medals
FROM Summer_Medals
WHERE Country = 'BRA' AND Medal = 'Gold' AND Year >= 1992
GROUP BY Year
ORDER BY Year;''', engine)

,Year,Medals
0,1992,13
1,1996,5
2,2004,18
3,2008,14
4,2012,14


# MAX Window function

In [69]:
pd.read_sql_query('''
WITH Brazil_Medals AS (SELECT Year, 
                              COUNT(*) AS Medals
                       FROM Summer_Medals
                       WHERE Country = 'BRA' AND Medal = 'Gold' AND Year >= 1992
                       GROUP BY Year
                       ORDER BY Year)
SELECT year, medals, 
       MAX(medals) OVER(ORDER BY year) AS Max_Medals
FROM Brazil_Medals;''', engine)

,Year,Medals,Max_Medals
0,1992,13,13
1,1996,5,13
2,2004,18,18
3,2008,14,18
4,2012,14,18


In [70]:
pd.read_sql_query('''
WITH Brazil_Medals AS (SELECT Year, 
                              COUNT(*) AS Medals
                       FROM Summer_Medals
                       WHERE Country = 'BRA' AND Medal = 'Gold' AND Year >= 1992
                       GROUP BY Year
                       ORDER BY Year)
SELECT Year, Medals,
       SUM(Medals) OVER (ORDER BY Year) AS Medals_RT
FROM Brazil_Medals;''', engine)

,Year,Medals,Medals_RT
0,1992,13,13
1,1996,5,18
2,2004,18,36
3,2008,14,50
4,2012,14,64


### Running totals of athlete medals
The running total (or cumulative sum) of a column helps you determine what each row's contribution is to the total sum.

* Return the `athlete`s, the number of `medals` they earned, and the medals running total, ordered by the `athlete`s' names in alphabetical order.

In [71]:
pd.read_sql_query('''
WITH Athlete_Medals AS (SELECT Athlete, 
                               COUNT(*) AS Medals
                        FROM Summer_Medals
                        WHERE Country = 'USA' AND Medal = 'Gold' AND Year >= 2000
                        GROUP BY Athlete)

SELECT Athlete,
       Medals,
       SUM(Medals) OVER (ORDER BY Athlete) AS Max_Medals
FROM Athlete_Medals
ORDER BY Athlete;''', engine)

,Athlete,Medals,Max_Medals
0,ABDUR-RAHIM Shareef,1,1
1,ABERNATHY Brent,1,2
2,ADRIAN Nathan,3,5
3,AHRENS Chris,1,6
4,AINSWORTH Kurt,1,7
...,...,...,...
362,WOLTERS Kara,1,513
363,WYLDE Peter,1,514
364,YOUNG Ernie,1,515
365,YOUNG Tim,1,516


### Maximum country medals by year

Getting the maximum of a country's earned medals so far helps you determine whether a country has broken its medals record by comparing the current year's earned medals and the maximum so far.

* Return the `year`, `country`, `medals`, and the maximum `medals` earned so far for each `country`, ordered by `year` in ascending order.

In [72]:
pd.read_sql_query('''
WITH Country_Medals AS (SELECT Year, 
                               Country, 
                               COUNT(*) AS Medals
                        FROM Summer_Medals
                        WHERE Country IN ('CHN', 'KOR', 'JPN') AND Medal = 'Gold' AND Year >= 2000
                        GROUP BY Year, Country)

SELECT year,
       country,
       medals,
       MAX(medals) OVER (PARTITION BY country ORDER BY year) AS Max_Medals
FROM Country_Medals
ORDER BY Country, Year;''', engine)

,Year,Country,Medals,Max_Medals
0,2000,CHN,39,39
1,2004,CHN,52,52
2,2008,CHN,74,74
3,2012,CHN,56,74
4,2000,JPN,5,5
5,2004,JPN,21,21
6,2008,JPN,23,23
7,2012,JPN,7,23
8,2000,KOR,12,12
9,2004,KOR,14,14


### Minimum country medals by year

So far, you've seen `MAX` and `SUM`, aggregate functions normally used with `GROUP BY`, being used as window functions. You can also use the other aggregate functions, like `MIN`, as window functions.

* Return the `year`, `medals` earned, and minimum medals earned so far.


In [73]:
pd.read_sql_query('''
WITH France_Medals AS (SELECT Year, COUNT(*) AS Medals
                       FROM Summer_Medals
                       WHERE Country = 'FRA' AND Medal = 'Gold' AND Year >= 2000
                       GROUP BY Year)

SELECT year, 
       medals, 
       MIN(medals) OVER(ORDER BY year) AS Min_Medals
FROM France_Medals
ORDER BY Year;''', engine)

,Year,Medals,Min_Medals
0,2000,22,22
1,2004,21,21
2,2008,25,21
3,2012,30,21


### Moving maximum of Scandinavian athletes' medals

Frames allow you to restrict the rows passed as input to your window function to a sliding window for you to define the start and finish.

Adding a frame to your window function allows you to calculate "moving" metrics, inputs of which slide from row to row.

* Return the `year`, `medals` earned, and the maximum medals earned, comparing only the current year and the next year.

In [74]:
pd.read_sql_query('''
WITH Scandinavian_Medals AS (SELECT Year, COUNT(*) AS Medals
                             FROM Summer_Medals
                             WHERE Country IN ('DEN', 'NOR', 'FIN', 'SWE', 'ISL') AND Medal = 'Gold'
                             GROUP BY Year)

SELECT year,
       medals,
       MAX(medals) OVER (ORDER BY year ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING) AS Max_Medals
FROM Scandinavian_Medals
ORDER BY Year;''', engine)

,Year,Medals,Max_Medals
0,1896,1,1
1,1900,1,77
2,1908,77,141
3,1912,141,159
4,1920,159,159
5,1924,48,48
6,1928,24,24
7,1932,17,17
8,1936,15,54
9,1948,54,54


### Moving maximum of Chinese athletes' medals

Frames allow you to "peek" forwards or backward <ins>without</ins> first using the relative fetching functions, `LAG` and `LEAD`, to fetch previous rows' values into the current row.

*  Return the `athlete`s, `medals` earned, and the maximum medals earned, comparing only the last two and current athletes, ordering by athletes' names in alphabetical order.

In [79]:
pd.read_sql_query('''
WITH Chinese_Medals AS (SELECT Athlete, COUNT(*) AS Medals
                        FROM Summer_Medals
                        WHERE Country = 'CHN' AND Medal = 'Gold' AND Year >= 2000
                        GROUP BY Athlete)

SELECT Athlete,
       Medals,
       MAX(Medals) OVER (ORDER BY Athlete ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Max_Medals
FROM Chinese_Medals
ORDER BY Athlete;''', engine)

,Athlete,Medals,Max_Medals
0,CAI Yalin,1,1
1,CAI Yun,1,1
2,CAO Lei,1,1
3,CAO Yuan,1,1
4,CHEN Ding,1,1
...,...,...,...
155,ZHOU Lulu,1,1
156,ZHOU Suhong,1,1
157,ZHU Qinan,1,1
158,ZOU Kai,5,5


# Moving Average

In [80]:
pd.read_sql_query('''SELECT Year, COUNT(*) AS Medals
                     FROM Summer_Medals
                     WHERE Country = 'USA'
                     AND Medal = 'Gold'
                     AND Year >= 1980
                     GROUP BY Year
                     ORDER BY Year;''', engine)

,Year,Medals
0,1984,168
1,1988,77
2,1992,89
3,1996,160
4,2000,130
5,2004,116
6,2008,125
7,2012,147


## Moving average for three periods

In [82]:
pd.read_sql_query('''
WITH US_Medals AS (SELECT Year, COUNT(*) AS Medals
                     FROM Summer_Medals
                     WHERE Country = 'USA'
                     AND Medal = 'Gold'
                     AND Year >= 1980
                     GROUP BY Year
                     ORDER BY Year)
SELECT Year, Medals,
       ROUND(AVG(Medals) OVER(ORDER BY Year ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) AS Medals_MA
       FROM US_Medals
       ORDER BY Year''', engine)

,Year,Medals,Medals_MA
0,1984,168,168.00
1,1988,77,122.50
2,1992,89,111.33
3,1996,160,108.67
4,2000,130,126.33
5,2004,116,135.33
6,2008,125,123.67
7,2012,147,129.33


### Moving average of Russian medals

Using frames with aggregate window functions allow you to calculate many common metrics, including moving averages and totals. These metrics track the change in performance over time.

* Calculate the 3-year moving average of medals earned

In [83]:
pd.read_sql_query('''
WITH Russian_Medals AS (SELECT Year, COUNT(*) AS Medals
                        FROM Summer_Medals
                        WHERE Country = 'RUS' AND Medal = 'Gold' AND Year >= 1980
                        GROUP BY Year)

SELECT Year, 
       Medals, 
       AVG(medals) OVER(ORDER BY Year ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Medals_MA
FROM Russian_Medals
ORDER BY Year;''', engine)

,Year,Medals,Medals_MA
0,1996,36,36.000000
1,2000,66,51.000000
2,2004,47,49.666667
3,2008,43,52.000000
4,2012,47,45.666667


### Moving total of countries' medals

What if your data is split into multiple groups spread over one or more columns in the table? Even with a defined frame, if you can't somehow separate the groups' data, one group's values will affect the average of another group's values.

* Calculate the 3-year moving sum of medals earned per country.

In [84]:
pd.read_sql_query('''
WITH Country_Medals AS (SELECT Year, Country, COUNT(*) AS Medals
                        FROM Summer_Medals
                        GROUP BY Year, Country)

SELECT Year, 
       Country, 
       Medals, 
       SUM(medals) OVER(PARTITION BY country ORDER BY Year ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Medals_MA
FROM Country_Medals
ORDER BY Country, Year;''', engine)

,Year,Country,Medals,Medals_MA
0,2012,None,4,4
1,2008,AFG,1,1
2,2012,AFG,1,2
3,1988,AHO,1,1
4,1984,ALG,2,2
...,...,...,...,...
1153,2004,ZIM,3,19
1154,2008,ZIM,4,23
1155,1896,ZZX,6,6
1156,1900,ZZX,34,40


# PIVOT -- CROSSTAB

`Before` table: Gold medals awarded to CHN, RUS, USA

```
| Country | Year | Awards |
|---------|------|--------|
| CHN     | 2008 | 74 |
| CHN     | 2012 | 56 |
| RUS     | 2008 | 43 |
| RUS     | 2012 | 47 |
| USA     | 2008 | 125 |
| USA     | 2012 | 147 |

```

### Pivoted by `year`

`After` table

```
| Country | 2008 | 2012 |
|---------|------|------|
| CHN     | 74 | 56 |
| RUS     | 43 | 47 |
| USA     | 125 | 147 |
```

#### สมมติเรามีข้อมูลตามแนวยาว แบบด้านล่างนี้ แล้วต้องการ pivot

In [166]:
pd.read_sql_query('''
SELECT Country, Year, 
       COUNT(*) AS Awards
FROM Summer_Medals
WHERE Country IN ('CHN', 'RUS', 'USA') AND Year IN (2008, 2012)AND Medal = 'Gold'
GROUP BY Country, Year
ORDER BY Country ASC, Year''', engine)

,Country,Year,Awards
0,CHN,2008,74
1,CHN,2012,56
2,RUS,2008,43
3,RUS,2012,47
4,USA,2008,125
5,USA,2012,147


### โดยปกติ คำตอบจะเป็นแบบนี้ ใช้ `CROSSTAB`

```
CREATE EXTENSION IF NOT EXISTS tablefunc;
SELECT * FROM CROSSTAB($$ SELECT Country, Year, COUNT(*) AS Awards
                          FROM Summer_Medals
                          WHERE Country IN ('CHN', 'RUS', 'USA') AND Year IN (2008, 2012) AND Medal = 'Gold'
                          GROUP BY Country, Year
                          ORDER BY Country ASC, Year ASC;
                          $$) AS ct (Country VARCHAR, "2008" INTEGER, "2012" INTEGER)
ORDER BY Country;
```
# แต่ SQLite ไม่มี `CROSSTAB` จึงต้องใช้การ pivot ด้วย `CASE` ตามด้านล่างนี้

In [185]:
pd.read_sql_query('''WITH cte AS (SELECT Country, Year, 
       COUNT(*) AS Awards
FROM Summer_Medals
WHERE Country IN ('CHN', 'RUS', 'USA') AND Year IN (2008, 2012)AND Medal = 'Gold'
GROUP BY Country, Year
ORDER BY Country ASC, Year)

SELECT Country,
       SUM(CASE WHEN year = 2008 THEN Awards ELSE 0 END) AS '2008',
       SUM(CASE WHEN year = 2012 THEN Awards ELSE 0 END) AS '2012'
FROM cte
GROUP BY country''', engine)

,Country,2008,2012
0,CHN,74,56
1,RUS,43,47
2,USA,125,147


# ต้องมี `SUM` นำหน้า สำหรับข้อมูลที่เป็นตัวเลข และต้องเป็น `MAX` นำหน้า สำหรับข้อมุล `STRING`

In [90]:
pd.read_sql_query('''
WITH Country_Awards AS (SELECT Country, Year, COUNT(*) AS Awards
                        FROM Summer_Medals
                        WHERE
                        Country IN ('CHN', 'RUS', 'USA')
                        AND Year IN (2004, 2008, 2012)
                        AND Medal = 'Gold' AND Sport = 'Gymnastics'
                        GROUP BY Country, Year
                        ORDER BY Country ASC, Year ASC)
SELECT Country, Year,
       CAST (RANK() OVER (PARTITION BY Year ORDER BY Awards DESC) AS INTEGER)  AS rank
FROM Country_Awards
ORDER BY Country, Year;''', engine)

,Country,Year,rank
0,CHN,2004,3
1,CHN,2008,1
2,CHN,2012,1
3,RUS,2004,1
4,RUS,2008,2
5,RUS,2012,2
6,USA,2004,2
7,USA,2008,3
8,USA,2012,3


### In SQLite ไม่มี CROSSTAB ต้อง pivot ด้วย CASE

* ปกติแล้วใช้ `CROSSTAB` แบบนี้

```
CREATE EXTENSION IF NOT EXISTS tablefunc;
SELECT * FROM CROSSTAB($$

WITH Country_Awards AS (SELECT Country, Year, COUNT(*) AS Awards
                        FROM Summer_Medals
                        WHERE
                        Country IN ('CHN', 'RUS', 'USA')
                        AND Year IN (2004, 2008, 2012)
                        AND Medal = 'Gold' AND Sport = 'Gymnastics'
                        GROUP BY Country, Year
                        ORDER BY Country ASC, Year ASC)
SELECT Country, Year,
       CAST (RANK() OVER (PARTITION BY Year ORDER BY Awards DESC) AS INTEGER)  AS rank
FROM Country_Awards
ORDER BY Country, Year;


$$) AS ct (Country VARCHAR,
"2004" INTEGER,
"2008" INTEGER,
"2012" INTEGER)
ORDER BY Country;
```

In [164]:
pd.read_sql_query('''
WITH Country_Awards AS (SELECT Country, Year, COUNT(*) AS Awards
                        FROM Summer_Medals
                        WHERE
                        Country IN ('CHN', 'RUS', 'USA')
                        AND Year IN (2004, 2008, 2012)
                        AND Medal = 'Gold' AND Sport = 'Gymnastics'
                        GROUP BY Country, Year
                        ORDER BY Country ASC, Year ASC),
                cte AS (SELECT Country, Year,
                               CAST (RANK() OVER (PARTITION BY Year ORDER BY Awards DESC) AS INTEGER)  AS rank
                        FROM Country_Awards
                        ORDER BY Country, Year)

SELECT Country,
       SUM(CASE WHEN year = 2004 THEN rank ELSE 0 END) AS '2004',
       SUM(CASE WHEN year = 2008 THEN rank ELSE 0 END) AS '2008',
       SUM(CASE WHEN year = 2012 THEN rank ELSE 0 END) AS '2012'
FROM cte
GROUP BY country;''', engine)

,Country,2004,2008,2012
0,CHN,3,1,1
1,RUS,1,2,2
2,USA,2,3,3


### A basic pivot
You have the following table of Pole Vault gold medalist countries by gender in 2008 and 2012.

```
| Gender | Year | Country |
|--------|------|---------|
| Men    | 2008 | AUS     |
| Men    | 2012 | FRA     |
| Women  | 2008 | RUS     |
| Women  | 2012 | USA     |
```


Pivot it by Year to get the following reshaped, cleaner table.

```
| Gender | 2008 | 2012 |
|--------|------|------|
| Men    | AUS  | FRA  |
| Women  | RUS  | USA  |
```

#### คำตอบด้วย CROSSTAB

```
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$ SELECT Gender, Year, Country
                          FROM Summer_Medals
                          WHERE Year IN (2008, 2012)
                            AND Medal = 'Gold'
                            AND Event = 'Pole Vault'
                          ORDER By Gender, Year;$$)  AS ct (Gender VARCHAR,
                                                            "2008" VARCHAR,
                                                            "2012" VARCHAR)

ORDER BY Gender;
```

In [182]:
pd.read_sql_query('''
WITH cte AS (SELECT Gender, Year, Country
             FROM Summer_Medals
             WHERE Year IN (2008, 2012)
             AND Medal = 'Gold' AND Event = 'Pole Vault'
             ORDER By Gender, Year)
             
SELECT Gender,
       MAX(CASE WHEN Year = 2008 THEN Country  END) AS '2008',
       MAX(CASE WHEN Year = 2012 THEN Country  END) AS '2012'
FROM cte
GROUP BY Gender''', engine)

,Gender,2008,2012
0,Men,AUS,FRA
1,Women,RUS,USA


ถ้าไม่มี `MAX` นำหน้า จะเอามาได้เฉพาะปีที่ตัวเลขปีต่ำสุด ดังนี้

In [186]:
pd.read_sql_query('''
WITH cte AS (SELECT Gender, Year, Country
             FROM Summer_Medals
             WHERE Year IN (2008, 2012)
             AND Medal = 'Gold' AND Event = 'Pole Vault'
             ORDER By Gender, Year)
             
SELECT Gender,
       CASE WHEN Year = 2008 THEN Country  END AS '2008',
       CASE WHEN Year = 2012 THEN Country  END AS '2012'
FROM cte
GROUP BY Gender''', engine)

,Gender,2008,2012
0,Men,AUS,None
1,Women,RUS,None


### Pivoting with ranking

You want to produce an easy scannable table of the rankings of the three most populous EU countries by how many gold medals they've earned in the 2004 through 2012 Olympic games. The table needs to be in this format:

```
| Country | 2004 | 2008 | 2012 |
|---------|------|------|------|
| FRA     | ...  | ...  | ...  |
| GBR     | ...  | ...  | ...  |
| GER     | ...  | ...  | ...  |
```

You'll need to count the gold medals each country has earned, produce the ranks of each country by medals earned, then pivot the table to this shape.

* Count the gold medals that France (`FRA`), the UK (`GBR`), and Germany (`GER`) have earned per country and year.

In [187]:
pd.read_sql_query('''
SELECT country,
       year,
       COUNT(*) AS Awards
FROM Summer_Medals
WHERE Country IN ('FRA', 'GBR', 'GER') AND Year IN (2004, 2008, 2012) AND Medal = 'Gold'
GROUP BY Country, Year
ORDER BY Country, Year''', engine)

,Country,Year,Awards
0,FRA,2004,21
1,FRA,2008,25
2,FRA,2012,30
3,GBR,2004,17
4,GBR,2008,31
5,GBR,2012,48
6,GER,2004,41
7,GER,2008,42
8,GER,2012,45


* Select the `country` and `year` columns, then rank the three countries by how many gold medals they earned per year.

In [191]:
pd.read_sql_query('''WITH Country_Awards AS (SELECT Country,
                               Year,
                               COUNT(*) AS Awards
                        FROM Summer_Medals
                        WHERE Country IN ('FRA', 'GBR', 'GER') AND Year IN (2004, 2008, 2012) AND Medal = 'Gold'
                        GROUP BY Country, Year)

SELECT Country,
       year,
       RANK() OVER(PARTITION BY year ORDER BY awards DESC) AS rank
FROM Country_Awards
ORDER BY Country ASC, Year ASC;''', engine)

,Country,Year,rank
0,FRA,2004,2
1,FRA,2008,3
2,FRA,2012,3
3,GBR,2004,3
4,GBR,2008,2
5,GBR,2012,1
6,GER,2004,1
7,GER,2008,1
8,GER,2012,2


* Pivot the query's results by `Year` by filling in the new table's correct column names.

```
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$ WITH Country_Awards AS (SELECT Country, Year, COUNT(*) AS Awards
                                                  FROM Summer_Medals
                                                  WHERE Country IN ('FRA', 'GBR', 'GER') AND Year IN (2004, 2008, 2012)
                                                  AND Medal = 'Gold'
                                                  GROUP BY Country, Year)

                          SELECT Country,
                                 Year,
                                 RANK() OVER(PARTITION BY Year ORDER BY Awards DESC) AS rank
                                 FROM Country_Awards
                                ORDER BY Country ASC, Year ASC;$$) AS ct (Country VARCHAR,
                                                                           "2014" INTEGER,
                                                                           "2008" INTEGER,
                                                                           "2012" INTEGER)

Order by Country ASC;
```

In [198]:
pd.read_sql_query('''
WITH Country_Awards AS (SELECT Country,
                               Year,
                               COUNT(*) AS Awards
                        FROM Summer_Medals
                        WHERE Country IN ('FRA', 'GBR', 'GER') AND Year IN (2004, 2008, 2012) AND Medal = 'Gold'
                        GROUP BY Country, Year),

                cte AS (SELECT Country,
                               year,
                               RANK() OVER(PARTITION BY year ORDER BY awards DESC) AS rank
                        FROM Country_Awards
                        ORDER BY Country, Year)
                        
SELECT Country,
       SUM(CASE WHEN year = 2004 THEN rank ELSE 0 END) AS '2004',
       SUM(CASE WHEN year = 2008 THEN rank ELSE 0 END) AS '2008',
       SUM(CASE WHEN year = 2012 THEN rank ELSE 0 END) AS '2012' 
FROM cte
GROUP BY Country''', engine)

,Country,2004,2008,2012
0,FRA,2,3,3
1,GBR,3,2,1
2,GER,1,1,2


### สมมติ ต้องการนับรวมจำนวนเหรียญทั้งหมด

In [213]:
pd.read_sql_query('''
SELECT Country, Medal, COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY Country, Medal
UNION 
SELECT Country, 'Total', COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY Country
ORDER BY Country, Medal
''', engine)

,Country,Medal,Awards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,CHN,Total,184
4,RUS,Bronze,56
5,RUS,Gold,43
6,RUS,Silver,44
7,RUS,Total,143


### ซึ่งตารางด้านบนมาจาก

In [209]:
pd.read_sql('''
SELECT Country, Medal, COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY Country, Medal''', engine)

,Country,Medal,Awards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,RUS,Bronze,56
4,RUS,Gold,43
5,RUS,Silver,44


In [210]:
pd.read_sql('''SELECT Country, 'Total', COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY Country
ORDER BY Country, Medal''', engine)

,Country,'Total',Awards
0,CHN,Total,184
1,RUS,Total,143


ใน SQLite ก็ต้องใช้วิธี `UNION` ทำ แต่ถ้าใน Postgre สามารถใช้ `ROLLUP` ได้ดังนี้

```
SELECT Country, Medal, COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY Country, ROLLUP(Medal)
ORDER BY Country, Medal;
```

### Country-level subtotals

You want to look at three Scandinavian countries' earned gold medals per country and gender in the year 2000. You're also interested in `Country`-level subtotals to get the total medals earned for each country, but `Gender`-level subtotals don't make much sense in this case, so disregard them.

* Count the gold medals awarded per country and gender.
* Generate `Country`-level gold award counts.

```
SELECT country,
       gender,
       COUNT(*) AS Gold_Awards
FROM Summer_Medals
WHERE Year = 2004 AND Medal = 'Gold' AND Country IN ('DEN', 'NOR', 'SWE')
-- Generate Country-level subtotals
GROUP BY country, ROLLUP(gender)
ORDER BY Country, Gender;


==================================
country	gender	gold_awards
==================================
DEN	    Men	     4
DEN	    Women	  15
DEN	    null	   19
NOR	    Men	     3
NOR	    Women	   2
NOR	    null	    5
SWE	    Men	     4
SWE	    Women  	 1
SWE	    null	    5
==================================
```



### All group-level subtotals

You want to break down all medals awarded to Russia in the 2012 Olympic games per gender and medal type. Since the medals all belong to one country, Russia, it makes sense to generate all possible subtotals (`Gender`- and `Medal`-level subtotals), as well as a grand total.

Generate a breakdown of the medals awarded to Russia per country and medal type, including all group-level subtotals and a grand total.

* Count the medals awarded per `gender` and `medal` type.
* Generate all possible group-level counts (per gender and medal type subtotals and the grand total).

```
SELECT gender,
       medal,
       COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2012 AND Country = 'RUS'
-- Get all possible group-level subtotals
GROUP BY CUBE(gender, medal)
ORDER BY Gender, Medal;


==================================
gender	medal	awards
----------------------------------
Men	   Bronze	34
Men	   Gold	  23
Men	   Silver	 7
Men	   null	  64
Women	 Bronze	17
Women	 Gold	  24
Women	 Silver	25
Women	 null	  66
null	  Bronze	51
null	  Gold	  47
null	  Silver	32
null	  null	  130
==================================

```

`NULL` ทั้งคู่ คือ grand totals

# COALESCE

* Useful when using SQL operations that return NULLs
* `ROLLUP` and `CUBE`
* `PIVOT`


### Cleaning up results

Returning to the breakdown of Scandinavian awards you previously made, you want to clean up the results by replacing the nulls with meaningful text.

* Turn the `null`s in the Country column to `All countries`, and the `null`s in the Gender column to `All genders`.

```
SELECT COALESCE(Country, 'All countries') AS Country,
       COALESCE(Gender, 'All genders') AS Gender,
       COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2004 AND Medal = 'Gold' AND Country IN ('DEN', 'NOR', 'SWE')
GROUP BY ROLLUP(Country, Gender)
ORDER BY Country, Gender;
```

ถ้าไม่ใช้ `COALESCE` จะเป็นงี้

```
==================================
country	gender	gold_awards
==================================
DEN	    Men	     4
DEN	    Women	  15
DEN	    null	   19
NOR	    Men	     3
NOR	    Women	   2
NOR	    null	    5
SWE	    Men	     4
SWE	    Women  	 1
SWE	    null	    5
null       null       29
==================================
```

หลังใช้ `COALESCE`

```
country	        gender    awards
All countries	 All genders	29
DEN	           All genders	19
DEN	           Men         	4
DEN	           Women	      15
NOR	           All genders	 5
NOR	           Men	         3
NOR	           Women	       2
SWE	           All genders	 5
SWE	           Men	         4
SWE	           Women	       1
```

### Summarizing results

After ranking each country in the 2000 Olympics by gold medals awarded, you want to return the top 3 countries in one row, as a comma-separated string. In other words, turn this:

```
| Country | Rank |
|---------|------|
| USA     | 1    |
| RUS     | 2    |
| AUS     | 3    |
| ...     | ...  |
```
into this:

`USA, RUS, AUS`

* Rank countries by the medals they've been awarded.

In [215]:
pd.read_sql_query('''
WITH Country_Medals AS (SELECT Country,
                               COUNT(*) AS Medals
                        FROM Summer_Medals
                        WHERE Year = 2000 AND Medal = 'Gold'
                        GROUP BY Country)

SELECT Country,
       RANK() OVER(ORDER BY Medals DESC) AS Rank
FROM Country_Medals
ORDER BY Rank''', engine)

,Country,Rank
0,USA,1
1,RUS,2
2,AUS,3
3,CHN,4
4,GER,5
5,NED,6
6,ROU,6
7,HUN,8
8,CUB,9
9,FRA,9


* Return the top 3 countries by medals awarded as one comma-separated string.

```
WITH Country_Medals AS (
  SELECT
    Country,
    COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE Year = 2000
    AND Medal = 'Gold'
  GROUP BY Country),

  Country_Ranks AS (
  SELECT
    Country,
    RANK() OVER (ORDER BY Medals DESC) AS Rank
  FROM Country_Medals
  ORDER BY Rank ASC)

-- Compress the countries column
SELECT STRING_AGG(country,', ')
FROM Country_Ranks
-- Select only the top three ranks
WHERE rank <= 3;
```